# Quiz 5

Estudiantes:

- Alonso Araya
- Pedro Soto
- Sofia Oviedo

# Load the data

In [ ]:
import pandas as pd
#data frame
sms_df = pd.read_table('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

display(sms_df.head())


# Install BERT

In [ ]:
%pip install transformers torch

# Login to HF

In [ ]:
from huggingface_hub import login

login(token = '')

# Load BERT

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device used: ", device)
model.to(device)

# Compute embeddings

In [ ]:
import torch

def compute_bert_embeddings(text_snippets, model):
    encoded_input = tokenizer(text_snippets, padding=True, truncation=True, return_tensors='pt')
    # Move encoded_input to the same device as the model
    encoded_input = {key: value.to(model.device) for key, value in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Use the embeddings of the [CLS] token
    embeddings = model_output.last_hidden_state[:, 0, :]
    #print("embeddings shape \n", embeddings.shape)
    return embeddings

sms_df['embeddings'] = sms_df['message'].apply(lambda x: compute_bert_embeddings([x], model).squeeze())

display(sms_df.head())

In [ ]:
#store the embedding in a tensor
embeddings_tensor = torch.stack(sms_df['embeddings'].tolist())
print("Embeddings tensor shape: ", embeddings_tensor.shape)
#convert the labels to numerical values in a tensor
label_mapping = {'ham': 0, 'spam': 1}
numerical_labels = sms_df['label'].map(label_mapping)
labels_tensor = torch.tensor(numerical_labels.tolist())
print("labels 0 ", (labels_tensor == 0).sum())
print("labels 1 ", (labels_tensor == 1).sum())

# Split the dataset (70% Train, 15% Validation, 15% Test)

In [ ]:
from sklearn.model_selection import train_test_split

# Primero dividir en 70% entrenamiento y 30% restante
X_train_full, X_temp, y_train_full, y_temp = train_test_split(
    embeddings_tensor, labels_tensor, test_size=0.30, random_state=42, stratify=labels_tensor
)

# Luego dividir el 30% restante en 15% validación y 15% prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Entrenamiento: {X_train_full.shape[0]} muestras ({X_train_full.shape[0]/len(embeddings_tensor)*100:.1f}%)")
print(f"Validación: {X_val.shape[0]} muestras ({X_val.shape[0]/len(embeddings_tensor)*100:.1f}%)")
print(f"Prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(embeddings_tensor)*100:.1f}%)")

# Verificar distribución de clases
print("\nDistribución de clases:")
print(f"Entrenamiento - Ham: {(y_train_full == 0).sum()}, Spam: {(y_train_full == 1).sum()}")
print(f"Validación - Ham: {(y_val == 0).sum()}, Spam: {(y_val == 1).sum()}")
print(f"Prueba - Ham: {(y_test == 0).sum()}, Spam: {(y_test == 1).sum()}")

# 1. (30 puntos) Para el dataset SMS_dataset disponible en implemente los siguientes modelos de clasificacion

```
a) Entrene una red neuronal con 2 variantes de arquitectura a definir por usted. Justifique las 2 variantes
y entrene ambos modelos, muestre sus curvas de aprendizaje para una particion de datos de
entrenamiento (70%) y validacion (15%) luego de la calibracion de los principales hiperparametros.
Evalue el error con una particion aleatoria de prueba (15%) Comente los resultados.
```

In [ ]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import seaborn as sns

## Modelos propuestos

In [ ]:
class SpamClassifierV1(nn.Module):
    def __init__(self, input_dim=768):
        super(SpamClassifierV1, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

class SpamClassifierV2(nn.Module):
    def __init__(self, input_dim=768):
        super(SpamClassifierV2, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

### Primer Modelo

Para el primer modelo propuesto "SpamClassifierV1", este representa el modelo mas profundo. Este busca reducir la dimensionalidad escalonada desde la entrada de 768 pasando por 512, 256, 128 y 1, esto puede ayudar a que el modelo aprenda representaciones mas complejas y reducir la dimensionalidad gradualmente puede evitar perde informacion abruptamente.

En el caso del dropout se utiliza valores de 0,5 y 0,3, esto permite regularizar correctamente el modelo y prevenir el overfitting, se reduce al final con 0,3 para poder preservar la mayor cantidad de informacion antes de realizar la clasificacion.

Esta utiliza cuatro capas y es capaz de poder capturar mejor los patrones complejos del set de datos y valores no lineales de los embeddings producidos por BERT. Sin embargo puede utilizar un poco mas de recursos al ser mas profundo.

### Segundo Modelo

En el segundo que se llama "SpamClassifierV2" tiene un perfil mas pequeño pero mas agresivo, ya que va desde la dimension de entrada de 768 hasta 256 y 64. Tiene tambien menos capas comparado con el modelo anterior y este va siendo un modelo mas pequeño con el fin de que sea mas eficiente a nivel computacional y comparar si existe un verdadero beneficio al usar algo mas escalonado y profundo como lo es el modelo V1, en contra de este modelo que es un poco mas pequeño y agresivo.

Ademas de eso tiene parametros como el dropout en 0,3 siendo un poco mas balanceado para perder menos informacion y consistente, esto puede ayudar a prevenir overfitting y preservar la mayor cantidad de informacion durante todo el modelo. Al ser mas pequeño tambien podria ayudar a que no se sobreajuste y puda generalizar con buenos resultados a un menor costo y complejidad, sin embargo al no ser tan profundo tiene el riesgo de caer en underfitting.

En general ambos utilizan la funcion de activacion ReLu conocida por ser mas eficiente a nivel computacional por su naturaleza linear y que puede ayudara mitigar el problema de gradientes desvanecientes, ayudando a que se puedan crear redes mas profundas, tambien tiene la ventaja dada sus propiedades no lineares ayuda a capturar estos mismos patrones en los datos.

Tambien se utiliza la función sigmoidal dado a que es una clasificacion binaria y nos puede dar resultados de 0 y 1, ademas en las proximas celdas de entrenamiento se utiliza BCELoss, que necesita de este formato para operar.

## Logica de Entrenamiento

In [ ]:
def train_model(model, train_loader, val_loader, epochs=50, lr=0.001, weight_decay=0.01):
    """
    Entrena un modelo de clasificación binaria para detección de spam con validación cruzada.

    Parameteros
    ----------
    model : torch.nn.Module
        Red neuronal a entrenar (SpamClassifierV1, SpamClassifierV2, etc.).
        Debe tener salida sigmoid para clasificación binaria.

    train_loader : torch.utils.data.DataLoader
        DataLoader con datos de entrenamiento. Debe retornar (features, labels)
        donde features son embeddings BERT (shape: [batch_size, 768]) y
        labels son enteros {0: ham, 1: spam}.

    val_loader : torch.utils.data.DataLoader
        DataLoader con datos de validación. Mismo formato que train_loader.
        Usado para evaluación independiente y selección del mejor modelo.

    epochs : int, optional, default=50
        Número máximo de épocas de entrenamiento. El entrenamiento puede
        detenerse antes si se detectan problemas de convergencia.

    lr : float, optional, default=0.001
        Learning rate para el optimizador Adam. Controla el tamaño de paso
        en la actualización de parámetros. Valores típicos: [0.0001, 0.01].

    weight_decay : float, optional, default=0.01
        Coeficiente de regularización L2 aplicado por el optimizador Adam.
        Previene sobreajuste penalizando parámetros grandes. Valores típicos: [0.0, 0.1].

    Retorna
    -------
    dict
        Diccionario con el historial completo de entrenamiento:

        - 'train_losses' : List[float]
            Pérdida BCE promedio por época en conjunto de entrenamiento.
        - 'val_losses' : List[float]
            Pérdida BCE promedio por época en conjunto de validación.
        - 'train_accuracies' : List[float]
            Accuracy (exactitud) por época en conjunto de entrenamiento.
        - 'val_accuracies' : List[float]
            Accuracy por época en conjunto de validación.
        - 'train_f1_scores' : List[float]
            F1-score por época en conjunto de entrenamiento.
        - 'val_f1_scores' : List[float]
            F1-score por época en conjunto de validación.
        - 'best_val_f1' : float
            Mejor F1-score alcanzado en validación durante todo el entrenamiento.

    """
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    train_f1_scores, val_f1_scores = [], []

    best_val_f1 = 0
    best_model_state = None

    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        train_loss = 0
        train_predictions = []
        train_targets = []

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = (outputs > 0.5).float()
            train_predictions.extend(predictions.cpu().numpy().flatten())
            train_targets.extend(batch_y.cpu().numpy().flatten())

        # Validación
        model.eval()
        val_loss = 0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device).float().unsqueeze(1)
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()

                predictions = (outputs > 0.5).float()
                val_predictions.extend(predictions.cpu().numpy().flatten())
                val_targets.extend(batch_y.cpu().numpy().flatten())

        # Calcular métricas
        train_acc = np.mean(np.array(train_predictions) == np.array(train_targets))
        val_acc = np.mean(np.array(val_predictions) == np.array(val_targets))

        train_f1 = f1_score(train_targets, train_predictions)
        val_f1 = f1_score(val_targets, val_predictions)

        # Guardar métricas
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        train_f1_scores.append(train_f1)
        val_f1_scores.append(val_f1)

        # Guardar mejor modelo
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = model.state_dict().copy()

        if (epoch + 1) % 10 == 0:
            print(f'Época {epoch+1}/{epochs}: '
                  f'Train Loss: {train_losses[-1]:.4f}, '
                  f'Val Loss: {val_losses[-1]:.4f}, '
                  f'Val F1: {val_f1:.4f}')

    # Restaurar mejor modelo
    model.load_state_dict(best_model_state)

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies,
        'train_f1_scores': train_f1_scores,
        'val_f1_scores': val_f1_scores,
        'best_val_f1': best_val_f1
    }

La funcion train_model permite entrenar un modelo, esta recibe como parametros el modelo, datos de entrenamiento y validacion, asi como el learning rate y regularizacion l2.

Esta funcion transfiere el modelo al device utilizado para asegurar que esta utilizando GPU. Se utiliza la funcion BCELoss que es una de las mas adecuadas para clasificacion binaria ya que mide la divergencia entre probabilidades que salieron del modelo y se ajusta perfectamente al modelo utilizar la funcion sigmoidal y utilizar etiquetas de 0 y 1.

Se utiliza Adam como optimizardor ya que puede ajustar adaptativamente parametros como el learning rate para los parametros, combina caracteristicas de RMSProp y SGS con momentum, asi como una convergencia mas estable y rapida en los modelos profundos, por lo que es una buena funcion segura y inicial por utilizar.

Tambien la funcion almacena todos las metricas como las perdidas de entrenamiento y validacion, asi como su accuracy y F1 Score para las particiones, finalmente evaluando cual fue el mejor modelo que obtuvo el mejor F1.

La funcion entrena el modelo en varios pasos, primero se limpian los gradientes con `zero_grad`, despues se hace un forward pass del modelo, se calculan las perdidas, se hace el calculo de gradiente y finalmente se optimizan los parametros. En general tambien se toma en cuenta pasar los valores al dispositivo ajustado para ser usado en el GPU o en caso de usar librerias como numpy se pasa a CPU. Durante el entrenamiento se convierten las probabilidades en predicciones binarias y se guardan para generar las metricas.

En cuanto a la fase de validacion se activa el modo por medio de `model.eval`, esta funcion desactiva el dropout y se ajusta para ser mas deterministico para la validacion. Por medio de `torch.grad` se desactiva el calculo de gradientes y solo se realiza forward pass, estos pasos ayudan a reducir el uso de recursos ya que no se necesitan nuevamente algunos pasos. Estas predicciones de las validaciones se van a guardar para el calculo de las metricas.

Se utilizo ademas un sistema de checkpointing el cual utiliza el F1 Score de la validacion y se guarda una copia del mejor modelo por el momento.

En el paso final se generan las metricas por medio de las predicciones y targets guardados en los pasos de entrenamiento y validacion, se generaron metricas de promedio de las perdidas, accuracy y F1 Score para el entrenamiento y validacion, todas estas son retornadas al final de la funcion.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Procesamiento en lotes y Gestion de Datos
batch_size = 32

train_dataset = TensorDataset(X_train_full, y_train_full)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Optimizacion de Parametros

In [ ]:
# Definir hiperparámetros a probar
learning_rates = [0.001, 0.0005]
weight_decays = [0.01, 0.001]
epochs = 50

best_params_v1 = {'lr': None, 'wd': None, 'score': 0}
best_params_v2 = {'lr': None, 'wd': None, 'score': 0}

print("\nCalibrando Modelo V1 (Red Profunda)...")
for lr in learning_rates:
    for wd in weight_decays:
        print(f"  Probando lr={lr}, weight_decay={wd}")
        model_v1 = SpamClassifierV1()
        history = train_model(model_v1, train_loader, val_loader,
                             epochs=30, lr=lr, weight_decay=wd)  # Menos épocas para calibración

        if history['best_val_f1'] > best_params_v1['score']:
            best_params_v1 = {'lr': lr, 'wd': wd, 'score': history['best_val_f1']}

print(f"  Mejor V1: lr={best_params_v1['lr']}, wd={best_params_v1['wd']}, F1={best_params_v1['score']:.4f}")

print("\nCalibrando Modelo V2 (Red Simple)...")
for lr in learning_rates:
    for wd in weight_decays:
        print(f"  Probando lr={lr}, weight_decay={wd}")
        model_v2 = SpamClassifierV2()
        history = train_model(model_v2, train_loader, val_loader,
                             epochs=30, lr=lr, weight_decay=wd)

        if history['best_val_f1'] > best_params_v2['score']:
            best_params_v2 = {'lr': lr, 'wd': wd, 'score': history['best_val_f1']}

print(f"  Mejor V2: lr={best_params_v2['lr']}, wd={best_params_v2['wd']}, F1={best_params_v2['score']:.4f}")

En la logica de arriba se realiza el entrenamiento en si de los modelos, para este efecto tambien se adecuo el codigo para poder realizar una optimizacion de parametros pequeña, se genero un rango de learning rates y weight decays a utilizar.

Se escogio ese rango de learning rate debido a que o.001 es estandar a utolizar con Adam y proporciona un buen balance, el valor de 0.0005 es un poco mas conservador y evita mucho mas el sobreajuste, estos dos valores proporcionan un punto medio para optimizar el modelo.

En cuanto al weight decay, el valor de 0.01 presenta una regularizacion un poco mas alta lo que podria evitar de gran manera el overfitting, en cuanto a 0.001 da mas flecibilidad al modelo y entre estos valores se puede variar el ajuste y la capacidad de generalizacion del modelo.

Esta funcion finalmente tiene como salida los mejores parametros por cada arquitectura, asi como sus F1 Score, tambien se puede observar las perdidas y F1 por cada grupo de epochs que paso, observando la evolucion del entrenamiento. Se guardan los mejores parametros para relizar el entrenamiento final con estas combinaciones por arquitectura.

In [ ]:
# Entrenar Modelo V1 con mejores parámetros
print("Entrenando Modelo V1 con mejores hiperparámetros...")
model_v1_final = SpamClassifierV1()
history_v1 = train_model(model_v1_final, train_loader, val_loader,
                        epochs=epochs,
                        lr=best_params_v1['lr'],
                        weight_decay=best_params_v1['wd'])

# Entrenar Modelo V2 con mejores parámetros
print("\nEntrenando Modelo V2 con mejores hiperparámetros...")
model_v2_final = SpamClassifierV2()
history_v2 = train_model(model_v2_final, train_loader, val_loader,
                        epochs=epochs,
                        lr=best_params_v2['lr'],
                        weight_decay=best_params_v2['wd'])

In [ ]:
plt.figure(figsize=(15, 10))

# Curvas de pérdida
plt.subplot(2, 3, 1)
plt.plot(history_v1['train_losses'], label='V1 Train', color='blue', alpha=0.7)
plt.plot(history_v1['val_losses'], label='V1 Val', color='blue', linestyle='--')
plt.plot(history_v2['train_losses'], label='V2 Train', color='red', alpha=0.7)
plt.plot(history_v2['val_losses'], label='V2 Val', color='red', linestyle='--')
plt.title('Curvas de Pérdida')
plt.xlabel('Época')
plt.ylabel('BCE Loss')
plt.legend()
plt.grid(True)

# Curvas de precisión
plt.subplot(2, 3, 2)
plt.plot(history_v1['train_accuracies'], label='V1 Train', color='blue', alpha=0.7)
plt.plot(history_v1['val_accuracies'], label='V1 Val', color='blue', linestyle='--')
plt.plot(history_v2['train_accuracies'], label='V2 Train', color='red', alpha=0.7)
plt.plot(history_v2['val_accuracies'], label='V2 Val', color='red', linestyle='--')
plt.title('Curvas de Precisión')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Curvas de F1-Score
plt.subplot(2, 3, 3)
plt.plot(history_v1['train_f1_scores'], label='V1 Train', color='blue', alpha=0.7)
plt.plot(history_v1['val_f1_scores'], label='V1 Val', color='blue', linestyle='--')
plt.plot(history_v2['train_f1_scores'], label='V2 Train', color='red', alpha=0.7)
plt.plot(history_v2['val_f1_scores'], label='V2 Val', color='red', linestyle='--')
plt.title('Curvas de F1-Score')
plt.xlabel('Época')
plt.ylabel('F1-Score')
plt.legend()
plt.grid(True)

# Comparación final de métricas
plt.subplot(2, 3, 4)
metrics = ['Train Acc', 'Val Acc', 'Train F1', 'Val F1']
v1_final_metrics = [history_v1['train_accuracies'][-1], history_v1['val_accuracies'][-1],
                   history_v1['train_f1_scores'][-1], history_v1['val_f1_scores'][-1]]
v2_final_metrics = [history_v2['train_accuracies'][-1], history_v2['val_accuracies'][-1],
                   history_v2['train_f1_scores'][-1], history_v2['val_f1_scores'][-1]]

x = np.arange(len(metrics))
width = 0.35
plt.bar(x - width/2, v1_final_metrics, width, label='Modelo V1', alpha=0.8)
plt.bar(x + width/2, v2_final_metrics, width, label='Modelo V2', alpha=0.8)
plt.title('Comparación Final de Métricas')
plt.ylabel('Score')
plt.xticks(x, metrics, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

En base al entrenamiento hecho con los parametros optimizados se generan las curvas de perdida, precision, F1 Score y comparacion entre ambos modelos. Esto nos permite ver de una mejor manera la evolucion de cada modelo durante los epochs.

En cuanto a la curva de perdida se puede observar como con pocos epochs rapidamente se reduce la perdida, ademas se observa una cierta estabilidad despues del epoch 10, las cuales se mantienen entre 0,2 y 0,4. Al ver las curvas de validacion estas tienden a estar cerca de las de entrenamiento, lo que dice que podrian tener un sobreajuste minimo y se obtiene una perdida baja lo que indica que el modelo tiene un buen ajuste.

En el accuracy se observa como antes de la mitad de los epochs realizados ya se obtiene resultados de 99%, el modelo V1 tiene una ligera ventaja sobre el V2 y se observa como despues del epoch 15 se estabilizan, asi como la diferencia entre entrenamiento y validacion es minima.

En los graficos de F1 Score, se muestra una historia similar, la cual alrededor del epoch 10-15 se empieza a estabilizar la puntuacio y rapidamente llegan a un punto alto de alrededor de 0,95, la arquitectura v1 sigue siendo la mejor pero con una ventaja minima y el valor del F1 Score es alto indicando un balance bueno entre precisión y recall. Al este ser alto y el accuracy tambien, se puede decir que el modelo maneja un buen rendimiento.

Al comparar ambas arquitecturas estras muestan varianzas minimas entre ellas.

## Evaluacion con el conjunto de prueba

In [ ]:
def evaluate_model(model, test_loader):
    """Evalúa el modelo en el conjunto de prueba"""
    model.eval()
    all_predictions = []
    all_targets = []
    all_probabilities = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            probabilities = outputs.cpu().numpy().flatten()
            predictions = (outputs > 0.5).float().cpu().numpy().flatten()
            targets = batch_y.cpu().numpy().flatten()

            all_predictions.extend(predictions)
            all_targets.extend(targets)
            all_probabilities.extend(probabilities)

    return np.array(all_predictions), np.array(all_targets), np.array(all_probabilities)

print("=== EVALUACIÓN EN CONJUNTO DE PRUEBA ===")

# Evaluar ambos modelos
pred_v1, true_labels, prob_v1 = evaluate_model(model_v1_final, test_loader)
pred_v2, _, prob_v2 = evaluate_model(model_v2_final, test_loader)

# Calcular métricas
f1_v1 = f1_score(true_labels, pred_v1)
f1_v2 = f1_score(true_labels, pred_v2)

accuracy_v1 = np.mean(pred_v1 == true_labels)
accuracy_v2 = np.mean(pred_v2 == true_labels)

print(f"\nRESULTADOS EN CONJUNTO DE PRUEBA:")
print(f"{'Modelo':<15} {'Accuracy':<10} {'F1-Score':<10}")
print("-" * 35)
print(f"{'V1 (Profunda)':<15} {accuracy_v1:<10.4f} {f1_v1:<10.4f}")
print(f"{'V2 (Simple)':<15} {accuracy_v2:<10.4f} {f1_v2:<10.4f}")

# Reportes detallados
print(f"\n=== REPORTE DETALLADO - MODELO V1 ===")
print(classification_report(true_labels, pred_v1, target_names=['Ham', 'Spam']))

print(f"\n=== REPORTE DETALLADO - MODELO V2 ===")
print(classification_report(true_labels, pred_v2, target_names=['Ham', 'Spam']))

Para la validacion final con los datos de prueba que fueron apartados anteriormente se desarrollo la funcion de evaluacion que se encarga de activar el modo de evaluacion en el modelo y se realizan las predicciones correspondientes sin el uso de gradientes, de esta iteracion se extraen todas las prediciones, probabilidades y los targets para la generacion de las metricas.

Se utiliza la funcion `evaluate_model` para evaluar los difeentes modelos por separados y realizar una comparacion correcta, se realiza el calculo del F1 Score y el accuracy asi como un `classification_report` para observar las diferentes metricas de precision, recall, f1, support y los mometos estadisticos como promedio macro, con peso y los calculos por cada clase.

En cuanto a los resultados finales, podemos observar como en general las dos arquitecturas tiene un rendimiento muy similar y la arquitectura profunda tiene una ventaja minima. Ambos tiene un accuracy de mas del 98% asi como un F1 Score mayor a 0.94.

Si nos vamos mas a fondo por cada clase, la etiqueta ham tiene unas metricas casi perfectas, esta clase es la que mas contenia datos para realizar pruebas. Estas metricas indican que los modelos pueden detectar todos los mensajes de ham, casi que nula clasificacion de spam como ham y cero diferencia en metricas. Estos resultados dicen que no existen falsos negativos para ham.

En cuanto a la clase de spam, esta contenia muchos menos datos para probar. En cuanto a esta clase si se observa un poco mas de diferencia en cuanto a las arquitecturas. En el modelo V1 se obtiene un mejor recall y menor precision, por lo que detecta mas spam pero genera un poco mas de falsos positivos, la red V2 genera menos falsos positivos al tener mejor precision y detecta menos spam real al tener un menor recall por lo que ambos tiene compromisos el uno al otro en esta prueba.

En general se obtiene que se detecta correctamente entre 90-95% de spam en ambos modelos, asi como un 5-10% de spam que se puede no detectar. En cuanto a mensajes clasificados como spam se obtiene un 97-99% de mensajes detectados como spam que realmente es spam y muy pocos falsos positivos 1-3%, esto para ambas arquitecturas.

En general para un tiempo de entrenamiento reducido y una red neuronal pequeñas se demuestra como se obtienen buenos valores, que no perfectos pero con compromisos minimos, lo que demuesta la capacidad de BERT para generar los embeddings, asi como arquitecturas simples que manejan de una manera satisfactoria el problema a un buen nivel, manejando un desbalance de clases correctamente, especialmente en un dataset tan desbalanceado.


In [ ]:
plt.figure(figsize=(12, 5))

# Matriz de confusión V1
plt.subplot(1, 2, 1)
cm_v1 = confusion_matrix(true_labels, pred_v1)
sns.heatmap(cm_v1, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title(f'Modelo V1 (Profunda)\nF1-Score: {f1_v1:.4f}')
plt.ylabel('Etiqueta Real')
plt.xlabel('Predicción')

# Matriz de confusión V2
plt.subplot(1, 2, 2)
cm_v2 = confusion_matrix(true_labels, pred_v2)
sns.heatmap(cm_v2, annot=True, fmt='d', cmap='Reds',
            xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title(f'Modelo V2 (Simple)\nF1-Score: {f1_v2:.4f}')
plt.ylabel('Etiqueta Real')
plt.xlabel('Predicción')

plt.tight_layout()
plt.show()

En estas matrices de confusion de apoyan los resultados en los cuales denota como ambas arquitecturas tienen muy pocos errores en comparacion con su exito por lo que es una buena solucion para el problema de deteccion de spam y aun con mucho mas espacio de mejora.

## (b) Evalue ambas arquitecturas previamente entrenadas, en 10 particiones aleatorias de entrenamiento y prueba, y reporte el F1-score promedio para ambas y su desviacion estandar. Comente los resultados.

## Entrenamiento con particiones diferentes

In [ ]:
print("=" * 80)
print("                    PARTE 1B: VALIDACIÓN CON 10 PARTICIONES ALEATORIAS")
print("=" * 80)

import time
from sklearn.model_selection import train_test_split
import pandas as pd

def train_and_evaluate_on_split(model_class, X, y, test_size=0.3, random_state=42,
                               best_params=None, epochs=40, verbose=False):
    """
    Entrena y evalúa un modelo de red neuronal en una partición específica.

    Parámetros
    ----------
    model_class : class
        Clase del modelo de PyTorch a instanciar (ej: SpamClassifierV1, SpamClassifierV2).
        Debe heredar de nn.Module y tener una salida con activación sigmoid para clasificación binaria.

    X : torch.Tensor
        Tensor de características de entrada con shape [n_samples, n_features].
        Típicamente embeddings BERT con dimensionalidad de 768.

    y : torch.Tensor
        Tensor de etiquetas binarias con shape [n_samples].
        Valores: 0 (ham) o 1 (spam) como enteros.

    test_size : float, opcional, default=0.3
        Proporción del dataset a usar para prueba. El resto se usa para entrenamiento.
        Debe estar entre 0.0 y 1.0.

    random_state : int, opcional, default=42
        Semilla para la división aleatoria de datos. Permite reproducibilidad de resultados
        entre diferentes ejecuciones con los mismos parámetros.

    best_params : dict, opcional, default=None
        Diccionario con los hiperparámetros optimizados del modelo:
        - 'lr' : float - Learning rate para el optimizador Adam
        - 'wd' : float - Coeficiente de weight decay (regularización L2)
        Si es None, se deben proporcionar valores por defecto.

    epochs : int, opcional, default=40
        Número de épocas de entrenamiento. Controla cuántas pasadas completas
        sobre los datos de entrenamiento realizará el modelo.

    verbose : bool, opcional, default=False
        Si True, imprime información de pérdida cada 10 épocas durante el entrenamiento.
        Útil para monitoreo del progreso en entrenamientos largos.

    Retorna
    -------
    tuple[float, float]
        Una tupla con las métricas de evaluación en el conjunto de prueba:
        - f1 : float
            F1-score del modelo en el conjunto de prueba. Métrica balanceada que
            combina precisión y recall, especialmente útil para datos desbalanceados.
        - accuracy : float
            Exactitud (accuracy) del modelo en el conjunto de prueba.
            Proporción de predicciones correctas sobre el total de muestras.
    """
    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Crear datasets y loaders
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Crear y entrenar modelo
    model = model_class()
    model = model.to(device)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(),
                          lr=best_params['lr'],
                          weight_decay=best_params['wd'])

    # Entrenamiento
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if verbose and (epoch + 1) % 10 == 0:
            print(f'    Época {epoch+1}/{epochs}: Loss: {epoch_loss/len(train_loader):.4f}')

    # Evaluación
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            predictions = (outputs > 0.5).float().cpu().numpy().flatten()
            targets = batch_y.cpu().numpy().flatten()

            all_predictions.extend(predictions)
            all_targets.extend(targets)

    # Calcular métricas
    f1 = f1_score(all_targets, all_predictions)
    accuracy = np.mean(np.array(all_predictions) == np.array(all_targets))

    return f1, accuracy

# Configuración para las 10 particiones
n_splits = 10
test_size = 0.3  # 70% entrenamiento, 30% prueba
epochs_per_split = 40

# Almacenar resultados
results_v1 = {'f1_scores': [], 'accuracies': [], 'times': []}
results_v2 = {'f1_scores': [], 'accuracies': [], 'times': []}

print(f"Evaluando ambos modelos en {n_splits} particiones aleatorias...")
print(f"Configuración: {100*(1-test_size):.0f}% entrenamiento, {100*test_size:.0f}% prueba, {epochs_per_split} épocas por partición")
print("-" * 80)

# Usar los mismos datos originales (embeddings_tensor, labels_tensor)
total_start_time = time.time()

for split in range(n_splits):
    print(f"\nPARTICIÓN {split + 1}/{n_splits}")

    # Usar diferentes seeds para cada partición
    random_seed = 42 + split * 10

    # Evaluar Modelo V1
    print("  Entrenando Modelo V1 (Red Profunda)...")
    start_time = time.time()
    f1_v1, acc_v1 = train_and_evaluate_on_split(
        SpamClassifierV1, embeddings_tensor, labels_tensor,
        test_size=test_size, random_state=random_seed,
        best_params=best_params_v1, epochs=epochs_per_split,
        verbose=False
    )
    time_v1 = time.time() - start_time

    results_v1['f1_scores'].append(f1_v1)
    results_v1['accuracies'].append(acc_v1)
    results_v1['times'].append(time_v1)

    # Evaluar Modelo V2
    print("  Entrenando Modelo V2 (Red Simple)...")
    start_time = time.time()
    f1_v2, acc_v2 = train_and_evaluate_on_split(
        SpamClassifierV2, embeddings_tensor, labels_tensor,
        test_size=test_size, random_state=random_seed,
        best_params=best_params_v2, epochs=epochs_per_split,
        verbose=False
    )
    time_v2 = time.time() - start_time

    results_v2['f1_scores'].append(f1_v2)
    results_v2['accuracies'].append(acc_v2)
    results_v2['times'].append(time_v2)

    # Mostrar resultados de esta partición
    print(f"  Resultados Partición {split + 1}:")
    print(f"    V1: F1={f1_v1:.4f}, Acc={acc_v1:.4f}, Tiempo={time_v1:.1f}s")
    print(f"    V2: F1={f1_v2:.4f}, Acc={acc_v2:.4f}, Tiempo={time_v2:.1f}s")

total_time = time.time() - total_start_time
print(f"\nEvaluación completada en {total_time:.1f} segundos")

Para esta seccion del entrenamiento con particiones se genero la funcion `train_and_evaluate_on_split` la cual recibe el modelo, la division de un 70/30% para el entrenamiento y pruebas, los epochs y si se genera output verbose. Esta funcion contiene logica para partir el dataset con estratificacion para mantener ambas clases y con el uso de semillas para poder repetir los resultados en multiples corridas. No se utiliza el conjunto de validacion ya que en este nos vamos a fijar en nada mas el rendimiento del modelo y utilizar el esquema original no es necesario ya que seria mas costoso y esto es mas para comparacion y referencia.

Esta seccion utiliza la misma lgica anterior la cual se crean los TensorDataset, se carga el modelo al GPU, se alista el BCELoss y el optimizador Adam, asi como el mismo proceso de entrenamiento con el paso de `zero_grad`, model, criterion, loss.backward y optimizer.step. La unica diferencia es que en este se corren todos los epochs y no hay early stopping.

Finalmente se almacenan las metricas de F1 Score, accuracy y tiempo de entrenamiento para cada una, esto con el fin de realizar un analisis comparativo mas adelante.

Se ejecutan el entrenamiento por arquitectura para las diez particiones necesarias y se van guardando las metricas de cada una.

## Resultados del entrenamiento con particiones

In [ ]:
print("\n" + "=" * 60)
print("                 ESTADÍSTICAS DE LAS 10 PARTICIONES")
print("=" * 60)

# Convertir a arrays numpy para facilitar cálculos
f1_scores_v1 = np.array(results_v1['f1_scores'])
f1_scores_v2 = np.array(results_v2['f1_scores'])
accuracies_v1 = np.array(results_v1['accuracies'])
accuracies_v2 = np.array(results_v2['accuracies'])

# Calcular estadísticas
stats_v1 = {
    'f1_mean': np.mean(f1_scores_v1),
    'f1_std': np.std(f1_scores_v1),
    'f1_min': np.min(f1_scores_v1),
    'f1_max': np.max(f1_scores_v1),
    'acc_mean': np.mean(accuracies_v1),
    'acc_std': np.std(accuracies_v1),
    'time_mean': np.mean(results_v1['times'])
}

stats_v2 = {
    'f1_mean': np.mean(f1_scores_v2),
    'f1_std': np.std(f1_scores_v2),
    'f1_min': np.min(f1_scores_v2),
    'f1_max': np.max(f1_scores_v2),
    'acc_mean': np.mean(accuracies_v2),
    'acc_std': np.std(accuracies_v2),
    'time_mean': np.mean(results_v2['times'])
}

# Mostrar resultados en tabla
print(f"\n{'Métrica':<20} {'Modelo V1':<15} {'Modelo V2':<15} {'Diferencia':<12}")
print("-" * 65)
print(f"{'F1-Score (μ ± σ)':<20} {stats_v1['f1_mean']:.4f} ± {stats_v1['f1_std']:.4f}  {stats_v2['f1_mean']:.4f} ± {stats_v2['f1_std']:.4f}  {abs(stats_v1['f1_mean'] - stats_v2['f1_mean']):.4f}")
print(f"{'F1-Score (min/max)':<20} {stats_v1['f1_min']:.4f} / {stats_v1['f1_max']:.4f}  {stats_v2['f1_min']:.4f} / {stats_v2['f1_max']:.4f}  -")
print(f"{'Accuracy (μ ± σ)':<20} {stats_v1['acc_mean']:.4f} ± {stats_v1['acc_std']:.4f}  {stats_v2['acc_mean']:.4f} ± {stats_v2['acc_std']:.4f}  {abs(stats_v1['acc_mean'] - stats_v2['acc_mean']):.4f}")
print(f"{'Tiempo promedio':<20} {stats_v1['time_mean']:.1f}s        {stats_v2['time_mean']:.1f}s        {abs(stats_v1['time_mean'] - stats_v2['time_mean']):.1f}s")

# Crear DataFrame para mejor visualización
results_df = pd.DataFrame({
    'Partición': range(1, n_splits + 1),
    'V1_F1': f1_scores_v1,
    'V2_F1': f1_scores_v2,
    'V1_Acc': accuracies_v1,
    'V2_Acc': accuracies_v2,
    'Diferencia_F1': f1_scores_v1 - f1_scores_v2
})

print(f"\nDETALLE POR PARTICIÓN:")
print(results_df.round(4))

Como se observan de los resultados mostrados por la celda anterior sobre las corridas de las diferentes particiones en ambas arquitecturas se muestran distintos puntos.

El F1 Score en la arquitectura V1 sigue siendo superior al llegar a mas de 0,96 en promedio y tener valores consistenes y muy similares, su accuracy tiene una diferencia menor en comparacion con la parte V2, ademas de individualmente presentar una mejora con los resultados iniciales. Al ver tambien los maximos y minimos la arquitectura V2 tiene un valor mas bajo en comparacion y la arquitectura V1 siempre es mas estable nunca bajando de 0,95.

En cuanto a la desviacion estandar del F1 Score se muestra que la arquitectura V1 es mucho mas estable y la V2 es casi dos veces mas dispersa, demostrando que la arquitectura V1 es mas estable.

En el accuracy podemos ver en su desviacion estandar y resultados por particion, se puede ver como los resultados son muy similares y no existe una diferencia destacable entre las dos, ambas presentan valores estables y su desviacion indica que los resultados no estaban muy dispersos, asimismos sus resultados de accuracy son de mas de 97% y no hay cambios drasticos entre corridas ni entre arquitecturas.

En cuanto al entrenamiento V2 es mas rapida por ser un poco mas simple pero tiene un costo de ser mas inestable y tener un poco de peores resultados, V1 sigue demostrando ser inclusive mejor que los resultados iniciales y los "peores" casos son de igual manera buenos por lo que es un buen candidato para la tarea y tiene una mejor capacidad de representacion.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Boxplot de F1-Scores
ax1 = axes[0, 0]
box_data = [f1_scores_v1, f1_scores_v2]
box_labels = ['Modelo V1\n(Red Profunda)', 'Modelo V2\n(Red Simple)']
bp = ax1.boxplot(box_data, labels=box_labels, patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax1.set_title('Distribución de F1-Scores\n(10 Particiones Aleatorias)')
ax1.set_ylabel('F1-Score')
ax1.grid(True, alpha=0.3)

# Agregar puntos individuales
for i, data in enumerate(box_data, 1):
    y = data
    x = np.random.normal(i, 0.04, size=len(y))
    ax1.scatter(x, y, alpha=0.6, s=20)

# 2. F1-Scores por partición
ax2 = axes[0, 1]
partitions = range(1, n_splits + 1)
ax2.plot(partitions, f1_scores_v1, 'o-', label='Modelo V1', color='blue', alpha=0.7)
ax2.plot(partitions, f1_scores_v2, 'o-', label='Modelo V2', color='red', alpha=0.7)
ax2.set_title('F1-Score por Partición')
ax2.set_xlabel('Número de Partición')
ax2.set_ylabel('F1-Score')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Histograma de diferencias
ax3 = axes[1, 0]
differences = f1_scores_v1 - f1_scores_v2
ax3.hist(differences, bins=6, alpha=0.7, color='green', edgecolor='black')
ax3.axvline(np.mean(differences), color='red', linestyle='--',
            label=f'Media: {np.mean(differences):.4f}')
ax3.set_title('Distribución de Diferencias\n(V1 - V2) en F1-Score')
ax3.set_xlabel('Diferencia en F1-Score')
ax3.set_ylabel('Frecuencia')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Comparación de medias con intervalos de confianza
ax4 = axes[1, 1]
models = ['Modelo V1', 'Modelo V2']
means = [stats_v1['f1_mean'], stats_v2['f1_mean']]
stds = [stats_v1['f1_std'], stats_v2['f1_std']]
colors = ['blue', 'red']

bars = ax4.bar(models, means, yerr=stds, capsize=5,
               color=colors, alpha=0.7, edgecolor='black')
ax4.set_title('F1-Score Promedio ± Desviación Estándar')
ax4.set_ylabel('F1-Score')
ax4.grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for bar, mean, std in zip(bars, means, stds):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + std + 0.005,
             f'{mean:.4f}±{std:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

En estas graficas se pueden visualizar mejor los resultados explicados anteriormente, se muestra la distribucion de F1 Score, por particion, promedio y desviacion estandar del mismo.

# 2. (20 puntos) Escoja al menos 2 modelos grandes del lenguaje, y uselos por medio del API de huggingface para hacer la clasificacion de los mensajes del dataset SMS_dataset, usando al menos una de las particiones de prueba anteriores. Reporte el prompt usado. Reporte el F1-score promedio para ambas y su desviacion estandar. Comente los resultados.

In [ ]:
%pip install transformers accelerate bitsandbytes --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

## Generacion de Prompt

In [ ]:
def create_spam_classification_prompt(message):
    """
    Crea un prompt estructurado para clasificación de spam usando el conocimiento del dominio.

    Parámetros:
    -----------
    message : str
        Mensaje SMS a clasificar

    Retorna:
    --------
    str : Prompt formateado para el LLM
    """
    prompt = f"""Task: Classify the following SMS message as either "spam" or "ham" (legitimate).

Guidelines:
- SPAM indicators: promotional content, prizes, urgent calls to action, suspicious links, money requests, lottery/contests
- HAM indicators: personal conversations, legitimate business communications, informational messages

SMS Message: "{message}"

Please only respond with the word"spam" or "ham" and nothing else, no other text or comments.

Classification: """

    return prompt

def extract_classification_from_response(response):
    """
    Extrae la clasificación de la respuesta del LLM.

    Parámetros:
    -----------
    response : str
        Respuesta completa del modelo

    Retorna:
    --------
    str : 'spam' o 'ham'
    """
    response_clean = response.lower().strip()

    # Buscar patrones de clasificación
    if 'spam' in response_clean and 'ham' not in response_clean:
        return 'spam'
    elif 'ham' in response_clean and 'spam' not in response_clean:
        return 'ham'
    elif response_clean.startswith('spam'):
        return 'spam'
    elif response_clean.startswith('ham'):
        return 'ham'
    else:
        # Se usa ham por defecto en caso de no clasificar
        print(f"Respuesta no clasificable: {response_clean}")
        return 'ham'

# Mostrar ejemplo del prompt utilizado
sample_msg = "URGENT! You've won £1000! Call 09061701461 to claim now!"
print("PROMPT UTILIZADO EN LA EVALUACIÓN:")
print("="*50)
print(create_spam_classification_prompt(sample_msg))
print("="*50)

## Logica de los Modelos y Funciones de Clasificacion

In [ ]:
class LLMSpamClassifier:
    """
    Clasificador de spam usando modelos de lenguaje.
    """

    def __init__(self, model_name, use_quantization=True):
        self.model_name = model_name
        self.use_quantization = use_quantization
        self.tokenizer = None
        self.model = None
        self.device = device

    def load_model(self):
        """Carga el modelo y tokenizer."""
        try:
            print(f"Cargando {self.model_name}...")

            # Cargar tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Cargar modelo
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=None,
                device_map="auto" if torch.cuda.is_available() else None,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                trust_remote_code=True
            )

            print(f"{self.model_name} cargado exitosamente")
            return True

        except Exception as e:
            print(f"Error cargando {self.model_name}: {str(e)}")
            return False

    def classify_single_message(self, message, max_length=400):
        """
        Clasifica un mensaje individual.

        Parámetros:
        -----------
        message : str
            Mensaje a clasificar
        max_length : int
            Longitud máxima del prompt

        Retorna:
        --------
        str : 'spam' o 'ham'
        """
        if self.model is None:
            raise ValueError("Modelo no cargado. Ejecuta load_model() primero.")

        prompt = create_spam_classification_prompt(message)

        # Tokenizar entrada
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            max_length=max_length,
            truncation=True,
            padding=True
        )

        # Mover tensores al dispositivo del modelo
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        # Generar respuesta
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=8,
                do_sample=False,
                temperature=0.1,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        # Decodificar solo la parte nueva de la respuesta
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )

        return extract_classification_from_response(response)

    def classify_messages_batch(self, messages, batch_size=4, show_progress=True):
        """
        Clasifica un lote de mensajes.

        Parámetros:
        -----------
        messages : list
            Lista de mensajes a clasificar
        batch_size : int
            Tamaño del lote (reducido para LLMs)
        show_progress : bool
            Mostrar barra de progreso

        Retorna:
        --------
        list : Lista de clasificaciones
        """
        predictions = []
        iterator = tqdm(range(0, len(messages), batch_size),
                       desc=f"Clasificando con {self.model_name.split('/')[-1]}") if show_progress else range(0, len(messages), batch_size)

        for i in iterator:
            batch = messages[i:i + batch_size]

            for message in batch:
                try:
                    pred = self.classify_single_message(message)
                    predictions.append(pred)
                except Exception as e:
                    print(f"Error en mensaje: {str(e)}")
                    predictions.append('ham')  # Clasificación segura por defecto

            # Pausa breve para gestión de memoria
            time.sleep(0.3)

        return predictions

## Configuracion de Modelos de Hugging Face

In [ ]:
llm_models_config = {
    "Llama-3-1B": {
        "model_name": "meta-llama/Llama-3.2-1B-Instruct",
        "use_quantization": False,
        "description": "Llama-3.2-1B-Instruct"
    },
    "Qwen-2-5-0.5B": {
        "model_name": "Qwen/Qwen2.5-0.5B-Instruct",
        "use_quantization": False,
        "description": "Qwen2.5-0.5B-Instruct"
    },
}

print("MODELOS LLM CONFIGURADOS PARA EVALUACIÓN:")
print("-" * 50)
for name, config in llm_models_config.items():
    print(f"• {name}: {config['model_name']}")
    print(f"  └─ {config['description']}")
print("-" * 50)

## Generación de Particiones y Entrenamiento

In [ ]:
def get_test_partition(split_index, max_test_samples=60):
    """
    Obtiene la misma partición de test usada en las evaluaciones de RN.

    Parámetros:
    -----------
    split_index : int
        Índice de la partición (0-9)
    max_test_samples : int
        Máximo de muestras para eficiencia con LLMs

    Retorna:
    --------
    tuple : (test_messages, test_labels_str, test_labels_numeric)
    """
    # Usar la misma lógica que train_and_evaluate_on_split
    random_seed = 42 + split_index * 10
    test_size = 0.3

    # Dividir datos con la misma estrategia
    original_indices = list(range(len(sms_df)))
    train_idx, test_idx = train_test_split(
        original_indices,
        test_size=test_size,
        random_state=random_seed,
        stratify=labels_tensor.numpy()
    )

    # Extraer mensajes y etiquetas del conjunto de prueba
    test_messages = [sms_df.iloc[idx]['message'] for idx in test_idx]
    test_labels_str = [sms_df.iloc[idx]['label'] for idx in test_idx]
    test_labels_numeric = [1 if label == 'spam' else 0 for label in test_labels_str]

    # Muestreo estratificado para eficiencia con LLMs
    if len(test_messages) > max_test_samples:
        # Mantener proporción de clases en la muestra
        stratified_sample_idx, _ = train_test_split(
            range(len(test_messages)),
            train_size=max_test_samples,
            random_state=42,
            stratify=test_labels_numeric
        )

        test_messages = [test_messages[i] for i in stratified_sample_idx]
        test_labels_str = [test_labels_str[i] for i in stratified_sample_idx]
        test_labels_numeric = [test_labels_numeric[i] for i in stratified_sample_idx]

    return test_messages, test_labels_str, test_labels_numeric

# Probar la función con una partición de ejemplo
test_msgs, test_lbls_str, test_lbls_num = get_test_partition(0)
print(f"Partición de prueba extraída:")
print(f"   • Total mensajes: {len(test_msgs)}")
print(f"   • Ham: {test_lbls_str.count('ham')} ({test_lbls_str.count('ham')/len(test_msgs)*100:.1f}%)")
print(f"   • Spam: {test_lbls_str.count('spam')} ({test_lbls_str.count('spam')/len(test_msgs)*100:.1f}%)")

print(f"\nEjemplos de mensajes:")
for i in range(min(3, len(test_msgs))):
    print(f"   [{test_lbls_str[i]}] {test_msgs[i][:70]}...")

In [ ]:
def evaluate_llm_multiple_partitions(classifier, n_splits=5, max_samples_per_partition=60):
    """
    Evalúa un clasificador LLM usando el mismo esquema de particiones que las redes neuronales.

    Parámetros:
    -----------
    classifier : LLMSpamClassifier
        Clasificador LLM inicializado y cargado
    n_splits : int
        Número de particiones (usar menos que RN por eficiencia)
    max_samples_per_partition : int
        Máximo de mensajes por partición

    Retorna:
    --------
    dict : Resultados con F1-scores y accuracies
    """
    f1_scores = []
    accuracies = []

    print(f"\nEvaluando {classifier.model_name.split('/')[-1]} en {n_splits} particiones")
    print(f"Usando máximo {max_samples_per_partition} mensajes por partición")

    for split in range(n_splits):
        print(f"\n--- Partición {split + 1}/{n_splits} ---")

        # Obtener datos de prueba para esta partición
        test_messages, test_labels_str, test_labels_numeric = get_test_partition(
            split, max_samples_per_partition
        )

        print(f"Evaluando {len(test_messages)} mensajes...")

        try:
            # Clasificar usando el LLM
            predictions_str = classifier.classify_messages_batch(test_messages, batch_size=3)

            # Convertir predicciones a formato numérico
            predictions_numeric = [1 if pred == 'spam' else 0 for pred in predictions_str]

            # Calcular métricas
            f1 = f1_score(test_labels_numeric, predictions_numeric, zero_division=0.0)
            accuracy = np.mean(np.array(predictions_numeric) == np.array(test_labels_numeric))

            f1_scores.append(f1)
            accuracies.append(accuracy)

            print(f"Resultados: F1={f1:.4f}, Accuracy={accuracy:.4f}")

            # Mostrar ejemplos de clasificación
            print("Ejemplos de clasificación:")
            correct_count = 0
            for i in range(min(4, len(predictions_str))):
                is_correct = predictions_str[i] == test_labels_str[i]
                if is_correct:
                    correct_count += 1
                status = "✅" if is_correct else "❌"
                print(f"   {status} Real: {test_labels_str[i]:4} | Pred: {predictions_str[i]:4} | {test_messages[i][:55]}...")

            print(f"   Precisión en ejemplos: {correct_count}/{min(4, len(predictions_str))}")

        except Exception as e:
            print(f"Error en partición {split + 1}: {str(e)}")
            f1_scores.append(0.0)
            accuracies.append(0.5)  # Random baseline

    # Calcular estadísticas finales
    stats = {
        'f1_scores': f1_scores,
        'accuracies': accuracies,
        'f1_mean': np.mean(f1_scores),
        'f1_std': np.std(f1_scores),
        'acc_mean': np.mean(accuracies),
        'acc_std': np.std(accuracies)
    }

    return stats

In [ ]:
print("="*80)
print("INICIANDO EVALUACIÓN DE MODELOS LLM")
print("="*80)

# Configuración de la evaluación
n_evaluation_splits = 4  # Reducido para eficiencia con LLMs
llm_results = {}

# Evaluar cada modelo LLM
for model_name, config in llm_models_config.items():
    print(f"\n{'='*60}")
    print(f"EVALUANDO: {model_name}")
    print(f"Modelo: {config['model_name']}")
    print(f"Descripción: {config['description']}")
    print(f"{'='*60}")

    # Inicializar clasificador
    classifier = LLMSpamClassifier(
        config["model_name"],
        config["use_quantization"]
    )

    # Cargar modelo
    model_loaded = classifier.load_model()

    if model_loaded:
        try:
            # Evaluar en múltiples particiones
            results = evaluate_llm_multiple_partitions(
                classifier,
                n_evaluation_splits
            )

            llm_results[model_name] = results

            print(f"\nRESULTADOS FINALES - {model_name}:")
            print(f"   F1-Score: {results['f1_mean']:.4f} ± {results['f1_std']:.4f}")
            print(f"   Accuracy: {results['acc_mean']:.4f} ± {results['acc_std']:.4f}")

        except Exception as e:
            print(f"Error durante evaluación: {str(e)}")
            # Resultados por defecto en caso de error
            llm_results[model_name] = {
                'f1_scores': [0.0] * n_evaluation_splits,
                'accuracies': [0.5] * n_evaluation_splits,
                'f1_mean': 0.0,
                'f1_std': 0.0,
                'acc_mean': 0.5,
                'acc_std': 0.0
            }

        # Limpieza de memoria GPU
        try:
            del classifier.model
            del classifier.tokenizer
            torch.cuda.empty_cache()
            print("Memoria GPU limpiada")
        except:
            pass

    else:
        print(f"No se pudo cargar el modelo {model_name}")
        # Resultados por defecto
        llm_results[model_name] = {
            'f1_scores': [0.0] * n_evaluation_splits,
            'accuracies': [0.5] * n_evaluation_splits,
            'f1_mean': 0.0,
            'f1_std': 0.0,
            'acc_mean': 0.5,
            'acc_std': 0.0
        }

## Resultados Finales

In [ ]:
print("="*80)
print("ANÁLISIS COMPARATIVO: LLMs vs REDES NEURONALES")
print("="*80)

# Crear tabla de resultados de LLMs
print("\nRESULTADOS DE MODELOS LLM:")
llm_results_df = pd.DataFrame({
    'Modelo LLM': list(llm_results.keys()),
    'F1-Score (μ)': [results['f1_mean'] for results in llm_results.values()],
    'F1-Score (σ)': [results['f1_std'] for results in llm_results.values()],
    'Accuracy (μ)': [results['acc_mean'] for results in llm_results.values()],
    'Accuracy (σ)': [results['acc_std'] for results in llm_results.values()]
})
print(llm_results_df.round(4).to_string(index=False))

# Comparación completa incluyendo redes neuronales
print(f"\nCOMPARACIÓN COMPLETA (LLMs vs Redes Neuronales):")
print("-" * 80)

# Crear datos para comparación
all_models_data = []

# Agregar resultados de redes neuronales (ya existentes)
all_models_data.append({
    'Modelo': 'Red Neuronal V1 (Profunda)',
    'Tipo': 'Red Neuronal + BERT',
    'F1-Score (μ)': stats_v1['f1_mean'],
    'F1-Score (σ)': stats_v1['f1_std'],
    'Accuracy (μ)': stats_v1['acc_mean'],
    'Accuracy (σ)': stats_v1['acc_std'],
    'Particiones': 10
})

all_models_data.append({
    'Modelo': 'Red Neuronal V2 (Simple)',
    'Tipo': 'Red Neuronal + BERT',
    'F1-Score (μ)': stats_v2['f1_mean'],
    'F1-Score (σ)': stats_v2['f1_std'],
    'Accuracy (μ)': stats_v2['acc_mean'],
    'Accuracy (σ)': stats_v2['acc_std'],
    'Particiones': 10
})

# Agregar resultados de LLMs
for model_name, results in llm_results.items():
    all_models_data.append({
        'Modelo': f'{model_name} (LLM)',
        'Tipo': 'LLM',
        'F1-Score (μ)': results['f1_mean'],
        'F1-Score (σ)': results['f1_std'],
        'Accuracy (μ)': results['acc_mean'],
        'Accuracy (σ)': results['acc_std'],
        'Particiones': n_evaluation_splits
    })

# Crear DataFrame completo
comparison_df = pd.DataFrame(all_models_data)
comparison_df_sorted = comparison_df.sort_values('F1-Score (μ)', ascending=False)

print(comparison_df_sorted.round(4).to_string(index=False))

# Identificar el mejor modelo
best_model = comparison_df_sorted.iloc[0]
print(f"\nMEJOR MODELO GENERAL:")
print(f"   {best_model['Modelo']}")
print(f"   F1-Score: {best_model['F1-Score (μ)']:.4f} ± {best_model['F1-Score (σ)']:.4f}")

# Análisis por tipo de modelo
print(f"\nANÁLISIS POR TIPO:")
type_analysis = comparison_df.groupby('Tipo').agg({
    'F1-Score (μ)': ['mean', 'max'],
    'Accuracy (μ)': ['mean', 'max']
}).round(4)
type_analysis.columns = ['F1_Mean_Avg', 'F1_Best', 'Acc_Mean_Avg', 'Acc_Best']
print(type_analysis)

print(f"\nRANKING FINAL POR F1-SCORE:")
for i, (_, row) in enumerate(comparison_df_sorted.iterrows(), 1):
    medal = "" if i == 1 else "" if i == 2 else "" if i == 3 else f"{i}."
    print(f"   {medal} {row['Modelo']}: {row['F1-Score (μ)']:.4f} ± {row['F1-Score (σ)']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo de gráficos
plt.style.use('default')
colors = ['#3498DB', '#E74C3C', '#2ECC71', '#F39C12', '#9B59B6', '#1ABC9C']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparación F1-Score de todos los modelos
ax1 = axes[0, 0]
models_short = [name.split()[0] + (' RN' if 'Red' in name else ' LLM' if 'LLM' in name else '')
                for name in comparison_df_sorted['Modelo']]
f1_means = comparison_df_sorted['F1-Score (μ)']
f1_stds = comparison_df_sorted['F1-Score (σ)']

bars1 = ax1.bar(range(len(models_short)), f1_means, yerr=f1_stds,
                capsize=5, color=colors[:len(models_short)],
                alpha=0.8, edgecolor='black', linewidth=1)

ax1.set_title('F1-Score: Comparación Todos los Modelos', fontsize=14, fontweight='bold')
ax1.set_ylabel('F1-Score')
ax1.set_xticks(range(len(models_short)))
ax1.set_xticklabels(models_short, rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')

# Agregar valores en las barras
for i, (bar, mean, std) in enumerate(zip(bars1, f1_means, f1_stds)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + std + 0.01,
             f'{mean:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 2. Comparación por tipo de modelo
ax2 = axes[0, 1]
type_comparison = comparison_df.groupby('Tipo')['F1-Score (μ)'].agg(['mean', 'std']).reset_index()
type_colors = ['#3498DB', '#E74C3C']

bars2 = ax2.bar(type_comparison['Tipo'], type_comparison['mean'],
                yerr=type_comparison['std'], capsize=5,
                color=type_colors, alpha=0.8, edgecolor='black')

ax2.set_title('F1-Score Promedio por Tipo', fontsize=14, fontweight='bold')
ax2.set_ylabel('F1-Score Promedio')
ax2.grid(True, alpha=0.3, axis='y')

for bar, mean, std in zip(bars2, type_comparison['mean'], type_comparison['std']):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + std + 0.01,
             f'{mean:.3f}', ha='center', va='bottom', fontweight='bold')

# 3. Scatter plot: Accuracy vs F1-Score
ax3 = axes[1, 0]
colors_scatter = ['blue' if 'Red' in modelo else 'red' for modelo in comparison_df['Modelo']]
scatter = ax3.scatter(comparison_df['Accuracy (μ)'], comparison_df['F1-Score (μ)'],
                     c=colors_scatter, alpha=0.7, s=150, edgecolors='black', linewidth=2)

for i, modelo in enumerate(comparison_df['Modelo']):
    label = modelo.split()[0] + (' RN' if 'Red' in modelo else ' LLM' if 'LLM' in modelo else '')
    ax3.annotate(label,
                (comparison_df.iloc[i]['Accuracy (μ)'], comparison_df.iloc[i]['F1-Score (μ)']),
                xytext=(8, 8), textcoords='offset points', fontsize=9, fontweight='bold')

ax3.set_xlabel('Accuracy')
ax3.set_ylabel('F1-Score')
ax3.set_title('Accuracy vs F1-Score', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Leyenda personalizada
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=12, label='Red Neuronal + BERT'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=12, label='LLM')
]
ax3.legend(handles=legend_elements, loc='lower right')

# 4. Distribución de F1-Scores (si hay datos suficientes)
ax4 = axes[1, 1]
if len(llm_results) > 0:
    # Datos para boxplot
    plot_data = []
    plot_labels = []

    # Agregar datos de RNs
    plot_data.extend([stats_v1['f1_mean']] * 10)  # Simular distribución
    plot_labels.extend(['RN V1'] * 10)
    plot_data.extend([stats_v2['f1_mean']] * 10)
    plot_labels.extend(['RN V2'] * 10)

    # Agregar datos de LLMs
    for model_name, results in llm_results.items():
        if len(results['f1_scores']) > 0:
            plot_data.extend(results['f1_scores'])
            plot_labels.extend([model_name] * len(results['f1_scores']))

    if plot_data:
        box_df = pd.DataFrame({'Modelo': plot_labels, 'F1_Score': plot_data})
        sns.boxplot(data=box_df, x='Modelo', y='F1_Score', ax=ax4)
        ax4.set_title('Distribución F1-Scores', fontsize=14, fontweight='bold')
        ax4.tick_params(axis='x', rotation=45)
        ax4.grid(True, alpha=0.3)
else:
    ax4.text(0.5, 0.5, 'Datos insuficientes\npara distribución',
             ha='center', va='center', transform=ax4.transAxes, fontsize=12)
    ax4.set_title('Distribución F1-Scores', fontsize=14)

plt.tight_layout()
plt.show()

# 3. (40 puntos) Compare los resultados entre los 4 modelos entrenados, y argumente ventajas y desventajas de cada uno con respecto a los resultados logrados.

In [ ]:
print("="*100)
print("PUNTO 3: ANÁLISIS COMPARATIVO COMPLETO DE LOS 4 MODELOS ENTRENADOS")
print("="*100)

# Recopilar todos los resultados
all_models_comparison = []

# Agregar resultados de redes neuronales
all_models_comparison.append({
    'Modelo': 'Red Neuronal V1 (Profunda)',
    'Tipo': 'Red Neuronal',
    'Arquitectura': '4 capas (768→512→256→128→1)',
    'F1_Mean': stats_v1['f1_mean'],
    'F1_Std': stats_v1['f1_std'],
    'Acc_Mean': stats_v1['acc_mean'],
    'Acc_Std': stats_v1['acc_std'],
    'Tiempo_Entrenamiento': stats_v1['time_mean'],
    'Particiones': 10,
    'Parametros_Aprox': '~2M parámetros de clasificador + BERT embeddings',
    'Complejidad': 'Alta',
    'Estabilidad': stats_v1['f1_std']
})

all_models_comparison.append({
    'Modelo': 'Red Neuronal V2 (Simple)',
    'Tipo': 'Red Neuronal',
    'Arquitectura': '3 capas (768→256→64→1)',
    'F1_Mean': stats_v2['f1_mean'],
    'F1_Std': stats_v2['f1_std'],
    'Acc_Mean': stats_v2['acc_mean'],
    'Acc_Std': stats_v2['acc_std'],
    'Tiempo_Entrenamiento': stats_v2['time_mean'],
    'Particiones': 10,
    'Parametros_Aprox': '~500K parámetros de clasificador + BERT embeddings',
    'Complejidad': 'Media',
    'Estabilidad': stats_v2['f1_std']
})

# Agregar resultados de LLMs
for model_name, results in llm_results.items():
    all_models_comparison.append({
        'Modelo': f'{model_name}',
        'Tipo': 'LLM',
        'Arquitectura': 'Transformer autoregresivo',
        'F1_Mean': results['f1_mean'],
        'F1_Std': results['f1_std'],
        'Acc_Mean': results['acc_mean'],
        'Acc_Std': results['acc_std'],
        'Tiempo_Entrenamiento': 'N/A (pre-entrenado)',
        'Particiones': n_evaluation_splits,
        'Parametros_Aprox': '3B+' if 'Qwen' in model_name else 'Variable',
        'Complejidad': 'Muy Alta',
        'Estabilidad': results['f1_std']
    })

# Crear DataFrame para análisis
comparison_df = pd.DataFrame(all_models_comparison)
comparison_df_sorted = comparison_df.sort_values('F1_Mean', ascending=False)

print("\nTABLA COMPARATIVA COMPLETA:")
print("-"*120)
display_cols = ['Modelo', 'Tipo', 'F1_Mean', 'F1_Std', 'Acc_Mean', 'Estabilidad', 'Complejidad']
print(comparison_df_sorted[display_cols].round(4).to_string(index=False))

print(f"\nRANKING DE RENDIMIENTO:")
for i, (_, row) in enumerate(comparison_df_sorted.iterrows(), 1):
    medal = "" if i == 1 else "" if i == 2 else "" if i == 3 else f""
    print(f"   {medal} {row['Modelo']}: F1={row['F1_Mean']:.4f}±{row['F1_Std']:.4f}")

# Análisis por categorías
print(f"\nANÁLISIS POR CATEGORÍAS:")
print("-"*60)

# Mejor rendimiento general
best_model = comparison_df_sorted.iloc[0]
print(f"MEJOR RENDIMIENTO GENERAL: {best_model['Modelo']}")
print(f"   • F1-Score: {best_model['F1_Mean']:.4f} ± {best_model['F1_Std']:.4f}")
print(f"   • Accuracy: {best_model['Acc_Mean']:.4f}")

# Más estable (menor desviación estándar)
most_stable = comparison_df.loc[comparison_df['Estabilidad'].idxmin()]
print(f"\nMÁS ESTABLE: {most_stable['Modelo']}")
print(f"   • Desviación F1: {most_stable['Estabilidad']:.4f}")
print(f"   • F1 promedio: {most_stable['F1_Mean']:.4f}")

# Análisis por tipo de modelo
type_analysis = comparison_df.groupby('Tipo').agg({
    'F1_Mean': ['mean', 'max', 'min'],
    'Acc_Mean': ['mean', 'max', 'min'],
    'Estabilidad': 'mean'
}).round(4)

print(f"\nANÁLISIS POR TIPO DE MODELO:")
print(type_analysis)

# Crear visualizaciones comparativas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Rendimiento F1 por modelo
ax1 = axes[0, 0]
models_short = [name.split()[0] + (' RN' if 'Red' in name else '') for name in comparison_df_sorted['Modelo']]
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
bars = ax1.bar(models_short, comparison_df_sorted['F1_Mean'],
               yerr=comparison_df_sorted['F1_Std'], capsize=5,
               color=colors[:len(models_short)], alpha=0.8, edgecolor='black')
ax1.set_title('F1-Score por Modelo', fontweight='bold')
ax1.set_ylabel('F1-Score')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Agregar valores en las barras
for bar, mean in zip(bars, comparison_df_sorted['F1_Mean']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{mean:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Estabilidad vs Rendimiento
ax2 = axes[0, 1]
scatter = ax2.scatter(comparison_df['Estabilidad'], comparison_df['F1_Mean'],
                     s=200, alpha=0.7,
                     c=['blue' if 'Red' in x else 'red' for x in comparison_df['Modelo']],
                     edgecolors='black', linewidth=2)
ax2.set_xlabel('Desviación Estándar F1 (menor = más estable)')
ax2.set_ylabel('F1-Score Promedio')
ax2.set_title('Estabilidad vs Rendimiento', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Agregar etiquetas
for i, modelo in enumerate(comparison_df['Modelo']):
    label = modelo.split()[0] + (' RN' if 'Red' in modelo else '')
    ax2.annotate(label,
                (comparison_df.iloc[i]['Estabilidad'], comparison_df.iloc[i]['F1_Mean']),
                xytext=(8, 8), textcoords='offset points', fontweight='bold')

# 3. Accuracy vs F1-Score
ax3 = axes[0, 2]
ax3.scatter(comparison_df['Acc_Mean'], comparison_df['F1_Mean'],
           s=200, alpha=0.7,
           c=['blue' if 'Red' in x else 'red' for x in comparison_df['Modelo']],
           edgecolors='black', linewidth=2)
ax3.set_xlabel('Accuracy Promedio')
ax3.set_ylabel('F1-Score Promedio')
ax3.set_title('Accuracy vs F1-Score', fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Comparación por tipo de modelo
ax4 = axes[1, 0]
type_means = comparison_df.groupby('Tipo')['F1_Mean'].mean()
type_stds = comparison_df.groupby('Tipo')['F1_Mean'].std().fillna(0)
bars = ax4.bar(type_means.index, type_means.values,
               yerr=type_stds.values, capsize=5,
               color=['#2E86AB', '#A23B72'], alpha=0.8)
ax4.set_title('F1-Score Promedio por Tipo', fontweight='bold')
ax4.set_ylabel('F1-Score Promedio')
ax4.grid(True, alpha=0.3)

# 5. Distribución de F1-Scores
ax5 = axes[1, 1]
rn_f1_scores = [stats_v1['f1_mean'], stats_v2['f1_mean']]
llm_f1_scores = [results['f1_mean'] for results in llm_results.values()]
ax5.boxplot([rn_f1_scores, llm_f1_scores], labels=['Redes Neuronales', 'LLMs'])
ax5.set_title('Distribución F1-Scores por Tipo', fontweight='bold')
ax5.set_ylabel('F1-Score')
ax5.grid(True, alpha=0.3)

# 6. Mapa de calor de métricas normalizadas
ax6 = axes[1, 2]
metrics_for_heatmap = comparison_df[['F1_Mean', 'Acc_Mean']].copy()
# Normalizar métricas (invertir estabilidad para que mayor sea mejor)
metrics_for_heatmap['Estabilidad_Inv'] = 1 / (comparison_df['Estabilidad'] + 0.001)
metrics_normalized = (metrics_for_heatmap - metrics_for_heatmap.min()) / (metrics_for_heatmap.max() - metrics_for_heatmap.min())

im = ax6.imshow(metrics_normalized.T, cmap='RdYlGn', aspect='auto')
ax6.set_xticks(range(len(comparison_df)))
ax6.set_xticklabels([name.split()[0] for name in comparison_df['Modelo']], rotation=45)
ax6.set_yticks(range(len(metrics_normalized.columns)))
ax6.set_yticklabels(['F1-Score', 'Accuracy', 'Estabilidad'])
ax6.set_title('Mapa de Calor de Métricas\n(Verde=Mejor, Rojo=Peor)', fontweight='bold')

plt.tight_layout()
plt.show()